In [27]:
### same as myProject_stage2_Knn, only including a few less features
### took Brett's LR code from branch 'LogisticRegression'
### adding my parts with marker $accode
### tried Knn

In [28]:
# Numpy for working with Arrays
import numpy as np
# Pandas for working with data tables
import pandas as pd
# SciPy implements many different numerical algorithms
import scipy as sp
import scipy.stats as stats
from scipy.sparse import csr_matrix
from scipy.sparse import hstack
# Module for plotting
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
# Module for pretty plotting
# import seaborn as sns
# Module for linear regression
import statsmodels.api as sm
import statsmodels.formula.api as smf

from patsy import dmatrices
from sklearn import linear_model, datasets
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn import cross_validation
from sklearn import preprocessing


In [29]:
### load data from csv files
water_values = pd.read_csv('./data/train_set_values.csv')
water_labels = pd.read_csv('./data/train_set_labels.csv')

#water_values.drop(['wpt_name', 'subvillage'], axis=1, inplace=True)

In [42]:
# list categorical features to be turned into dummy variables:
##$accodechange excluding wpt_name
"""
cat_features = ['region_code', 'district_code', 'basin', 'region', 'public_meeting', \
                'scheme_management', 'permit', 'extraction_type', 'extraction_type_group', 'extraction_type_class', \
                'management', 'payment', 'water_quality', 'quantity', \
                'source', 'waterpoint_type', 'date_recorded', 'recorded_by', 'funder', 'installer', \
               'lga', 'ward', 'scheme_name', 'management_group', 'wpt_name', 'subvillage', \
               'payment_type', 'quality_group', 'quantity_group', 'source_class', 'source_type', 'waterpoint_type_group']

"""
cat_features = ['region_code', 'district_code', 'basin', 'region', 'public_meeting', \
                'scheme_management', 'permit', 'extraction_type', 'extraction_type_group', 'extraction_type_class', \
                'management', 'payment', 'water_quality', 'quantity', \
                'source', 'waterpoint_type', 'date_recorded', 'recorded_by', 'funder', 'installer', \
               'lga', 'ward', 'scheme_name', 'management_group', 'subvillage', \
               'payment_type', 'quality_group', 'quantity_group', 'source_class', 'source_type', 'waterpoint_type_group']


# make dataframe of just the categorical features
water_cat = water_values[cat_features]

# make dataframe of just numeric (basically, the rest of the columns)
water_num = water_values[list(set(water_values.columns) - set(cat_features))]
water_num=water_num.ix[:,water_num.columns!='wpt_name'] ##$accodechange excluding wpt_name

In [57]:
# prepare test data for model prediction
test = pd.read_csv('data/test_set_values.csv')

#test['status_group'] = np.random.choice(range(1, 3), test.shape[0])
#test.drop(['wpt_name', 'subvillage'], axis=1, inplace=True)

# make dataframe of just the categorical features
test_cat = test[cat_features]

# make dataframe of just numeric (basically, the rest of the columns)
test_num = test[list(set(test.columns) - set(cat_features))]
test_num=test_num.ix[:,test_num.columns!='wpt_name']  ##$accodechange excluding wpt_name

# replace 0s with NaN
for col in ['num_private', 'amount_tsh', 'population', 'construction_year', 'gps_height']:
    test_num.loc[:,col].replace(0, np.nan, inplace = True)



In [58]:
all_cat = pd.concat([water_cat, test_cat])
all_num = pd.concat([water_num, test_num])

In [59]:
# convert status labels to numeric
water_labels['status_group'] = water_labels.status_group.factorize()[0]

In [60]:
# replace 0s with NaN
for col in ['num_private', 'amount_tsh', 'population', 'construction_year', 'gps_height']:
    all_num.loc[:,col].replace(0, np.nan, inplace = True)

In [61]:
# preprocessing steps
# encode labels to floats
le = preprocessing.LabelEncoder()

# make dummy variables
enc = preprocessing.OneHotEncoder(handle_unknown = 'ignore')

In [62]:
for col in cat_features:
    all_cat.loc[:,col] = le.fit_transform(all_cat.loc[:,col])

In [63]:
all_cat = enc.fit_transform(all_cat[cat_features])
all_cat

<74250x31662 sparse matrix of type '<type 'numpy.float64'>'
	with 2301750 stored elements in Compressed Sparse Row format>

In [64]:
water_num.columns

Index([u'gps_height', u'longitude', u'latitude', u'amount_tsh', u'num_private',
       u'construction_year', u'id', u'population'],
      dtype='object')

In [65]:
# impute data
imp = preprocessing.Imputer(missing_values='NaN', strategy='most_frequent', axis=0, verbose=0, copy=True)
all_num = imp.fit_transform(all_num)

In [66]:
# standardize numeric variables
all_num = preprocessing.scale(all_num)

In [67]:
water_cat = all_cat[:59400,:]
water_cat.shape #$accodechange was 77346 features, reduced to  31662

(59400, 31662)

In [68]:
test_cat = all_cat[59400:,:]
test_cat.shape

(14850, 31662)

In [69]:
all_cat.shape

(74250, 31662)

In [70]:
all_num.shape

(74250, 8)

In [71]:
water_num = all_num[:59400,:]
water_num.shape

(59400, 8)

In [72]:
test_num = all_num[59400:,:]
test_num.shape

(14850, 8)

In [73]:
# convert to sparse matrix
water_num = csr_matrix(water_num)

In [74]:
# concatenate 
X = hstack([water_cat, water_num])

In [75]:
# flatten label column into a 1-D array called y
y = np.ravel(water_labels['status_group'])

In [26]:
### $accode - running Knn instead
# instantiate a logistic regression model, and fit with X and y
#model = LogisticRegression(penalty = 'l2')
#model = model.fit(X, y)

# check the accuracy on the training set
#model.score(X, y)

In [83]:
### $accode - try Knn
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=3)
model.fit(X,y)
#model.score(X,y) #$accode commented out this line because kernel keeps failing; from this line the code stayed the same
#err = (y_test != est.predict(X_test)).mean()

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')

In [43]:
# create prediction accuracy function to be used in cross val
def predict_accuracy(model, x,y):
    y_hat = model.predict(x)
    return np.mean(y_hat == y)

# run cross validation
score = cross_validation.cross_val_score(model, X, y, scoring = predict_accuracy).mean()

score

0.78750841750841749

In [34]:
#print "Features sorted by their score:"
#print sorted(zip(map(lambda x: round(x, 4), model.scores_),                 names), reverse=True)

In [84]:
# create train, test sets for X,Y to cross-validate, k=1 (just to verify above cross-val)
xtrain, xtest, ytrain, ytest = \
cross_validation.train_test_split(X, y, test_size=0.2, random_state=11)

# verify shapes
print xtrain.shape, ytrain.shape
print xtest.shape, ytest.shape

# predict labels for test set
yhat = model.predict(xtest)

# calculate accuracy
np.mean(yhat == ytest)

"""
was 
(47520, 77354) (47520,)
(11880, 77354) (11880,)

with no wpt_name became

(47520, 31670) (47520,)
(11880, 31670) (11880,)
"""

(47520, 31670) (47520,)
(11880, 31670) (11880,)


'\nwas \n(47520, 77354) (47520,)\n(11880, 77354) (11880,)\n\nwith no wpt_name became\n\n(47520, 31670) (47520,)\n(11880, 31670) (11880,)\n'

In [30]:
'''
# prepare test data for model prediction
test = pd.read_csv('./test_set_values.csv')

#test['status_group'] = np.random.choice(range(1, 3), test.shape[0])
test.drop(['id', 'wpt_name', 'subvillage'], axis=1, inplace=True)

# make dataframe of just the categorical features
test_cat = test[cat_features]

# make dataframe of just numeric (basically, the rest of the columns)
test_num = test[list(set(test.columns) - set(cat_features))]

# replace 0s with NaN
for col in ['num_private', 'amount_tsh', 'population', 'construction_year', 'gps_height']:
    test_num.loc[:,col].replace(0, np.nan, inplace = True)

# encode categorical features w/ floats
for col in cat_features:
    test_cat.loc[:,col] = le.transform(test_cat.loc[:,col])

# make dummy variables
test_cat = enc.transform(test_cat[cat_features])

# impute data
test_num = imp.transform(test_num)

# standardize numeric variables
test_num = preprocessing.scale(test_num)
'''


"\n# prepare test data for model prediction\ntest = pd.read_csv('./test_set_values.csv')\n\n#test['status_group'] = np.random.choice(range(1, 3), test.shape[0])\ntest.drop(['id', 'wpt_name', 'subvillage'], axis=1, inplace=True)\n\n# make dataframe of just the categorical features\ntest_cat = test[cat_features]\n\n# make dataframe of just numeric (basically, the rest of the columns)\ntest_num = test[list(set(test.columns) - set(cat_features))]\n\n# replace 0s with NaN\nfor col in ['num_private', 'amount_tsh', 'population', 'construction_year', 'gps_height']:\n    test_num.loc[:,col].replace(0, np.nan, inplace = True)\n\n# encode categorical features w/ floats\nfor col in cat_features:\n    test_cat.loc[:,col] = le.transform(test_cat.loc[:,col])\n\n# make dummy variables\ntest_cat = enc.transform(test_cat[cat_features])\n\n# impute data\ntest_num = imp.transform(test_num)\n\n# standardize numeric variables\ntest_num = preprocessing.scale(test_num)\n"

In [85]:
# convert to sparse matrix
test_num = csr_matrix(test_num)

# concatenate 
Xtest = hstack([test_cat, test_num])

In [86]:
y_hat = model.predict(Xtest)
y_hat

array([0, 0, 0, ..., 0, 0, 1])

In [30]:
len(y_hat)

14850

In [31]:
len(test)

14850

In [87]:
# submission

test['status_group'] = y_hat

map_dict = {0: 'functional', 1: 'non functional', 2: 'functional needs repair'}
test['status_group'] = test['status_group'].map(map_dict)
test['status_group']

0                     functional
1                     functional
2                     functional
3                 non functional
4                     functional
5                     functional
6                     functional
7                 non functional
8                 non functional
9                     functional
10                    functional
11                non functional
12                non functional
13                non functional
14                    functional
15                    functional
16                    functional
17                    functional
18                    functional
19                non functional
20                non functional
21                    functional
22                non functional
23                non functional
24                    functional
25                    functional
26                non functional
27                non functional
28       functional needs repair
29                    functional
          

In [88]:
test_logreg = test[['id', 'status_group']]
test_logreg.to_csv('test_knn3_noFeature_wptname.csv', index = False)

"""
submission notes:
Knn 1, no wptname, accuracy .7646
Knn 3, no wptname, accuracy .7855
"""

'\nsubmission notes:\nKnn 1, no wptname, accuracy 7646\n'

In [37]:
##### random code chunks below this line #####

In [38]:
###############################################

In [39]:
a = pd.read_csv('./test_logreg.csv')
b = pd.read_csv('./test_logreg2.csv')


IOError: File ./test_logreg.csv does not exist

In [ ]:
# received error that test dataset does not have as many columns as training (3dummy columns missing)
# this is to find out which ones
mask = np.in1d(X.columns, Xtest.columns)
print np.where(~mask)[0]



In [ ]:
water_unique = water_cat.apply(lambda x: len(x.unique()))
print water_unique.sort_values()